<a href="https://colab.research.google.com/github/adamserag1/Interpretability-for-VRDU-models/blob/main/finetuning/LayoutLM3-RVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
import torch
import torch.nn as nn
from transformers import LayoutLMv3ForSequenceClassification, AutoProcessor
from PIL import Image,ImageDraw, ImageFont
from datasets import load_dataset, load_from_disk
import pandas as pd
import evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [4]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -r /content/drive/MyDrive/THESIS/rvl_cdip_financial_subset /content

In [6]:
model = LayoutLMv3ForSequenceClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                            num_labels=5,
                                                            id2label={0: "form", 1: "invoice", 2: "budget", 3: "file folder", 4: "questionnaire"},
                                                            label2id={"form": 0, "invoice": 1, "budget": 2, "file folder": 3, "questionnaire": 4})

def freeze_layoutlmv3_layers(model, num_encoder_layers_to_freeze: int = 6, freeze_embeddings: bool = True):
    """
    Args:
        model  : LayoutLMv3ForSequenceClassification
        num_encoder_layers_to_freeze : freeze layers 0 … N-1
        freeze_embeddings            : whether to freeze word/patch/pos embeddings
    """
    # 1) embeddings
    if freeze_embeddings:
        for p in model.layoutlmv3.embeddings.parameters():
            p.requires_grad = False

    # 2) first N encoder layers
    for layer in model.layoutlmv3.encoder.layer[:num_encoder_layers_to_freeze]:
        for p in layer.parameters():
            p.requires_grad = False

    # utility print
    n_total = sum(p.numel() for p in model.parameters())
    n_train = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Froze embeddings={freeze_embeddings}, "
          f"layers 0-{num_encoder_layers_to_freeze-1}. "
          f"Trainable params: {n_train/1e6:.1f} M / {n_total/1e6:.1f} M")

freeze_layoutlmv3_layers(model, num_encoder_layers_to_freeze=6, freeze_embeddings=True)


model.to(device)

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Froze embeddings=True, layers 0-5. Trainable params: 43.9 M / 125.9 M


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [7]:
def normalize_bbox(bbox, width, height):
    """
    Convert absolute pixel coords (x0,y0,x1,y1) to 0-1000 scale *and*
    clamp every value into [0, 1000].  If the bbox already looks
    normalised, leave it unchanged.
    """
    if max(bbox) <= 1000:
        # assume already in 0-1000 layout units
        x0, y0, x1, y1 = bbox
    else:
        x0, y0, x1, y1 = bbox
        x0 = int(1000 * x0 / width)
        y0 = int(1000 * y0 / height)
        x1 = int(1000 * x1 / width)
        y1 = int(1000 * y1 / height)

    # hard clamp
    return [
        max(0, min(1000, x0)),
        max(0, min(1000, y0)),
        max(0, min(1000, x1)),
        max(0, min(1000, y1)),
    ]

In [8]:
def encode(batch):
  # Not a batch in the end
  # print(batch["image"])
  # print(batch.keys())

  images = [sample.convert('RGB') for sample in batch["image"]]
  labels = batch["label"]
  words = batch["words"]
  # normalize bboxes
  boxes = []
  for image, bboxes in zip(images, batch["bboxes"]):
    w, h = image.size
    temp = []
    for idx, _ in enumerate(bboxes):
      bboxes[idx] = normalize_bbox(bboxes[idx], w, h)
      temp.append(bboxes[idx])
    boxes.append(temp)

  encoding = processor(
      images,
      words,
      boxes=boxes,
      truncation=True,
      padding="max_length",
      max_length=512,
      return_tensors="pt"
  )
  encoding["labels"] = torch.tensor(labels, dtype=torch.long)
  for k in encoding:
      encoding[k] = encoding[k].to(device)

  return encoding

In [9]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [10]:
rvl = load_from_disk("/content/rvl_cdip_financial_subset")
dataset_split = rvl.train_test_split(test_size=0.2, shuffle=True, seed=42)
train = dataset_split["train"]
val = dataset_split["test"]
print(len(train))
print(len(val))
train_dataset = train.map(encode, batched=True, remove_columns=train.column_names, batch_size=8)
val_dataset = val.map(encode, batched=True, remove_columns=val.column_names, batch_size=8)

4000
1000


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
       early_stopping_patience=2,
   )

In [12]:
from huggingface_hub import login
login()

In [13]:
from transformers import TrainingArguments, Trainer, DefaultDataCollator
# training_args = TrainingArguments(
#     output_dir="./layoutlmv3-docclass-finetuned-2.0",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=1e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=10,                 # Enough to converge on 5-class task
#     weight_decay=0.01,
#     # warmup_ratio=0.1,
#     logging_dir="./logs",
#     logging_strategy="steps",
#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_accuracy",
#     save_total_limit=2,
#     seed=42,
#     fp16=True,
#     push_to_hub=True,
#     report_to = "none"
# )
training_args = TrainingArguments(
    output_dir               = "./layoutlmv3-docclass-finetuned-frz",
    # — schedule & monitoring —
    eval_strategy      = "steps",
    eval_steps               = 250,            # ≈ two evals per epoch
    save_strategy            = "steps",
    save_steps               = 250,
    load_best_model_at_end   = True,
    metric_for_best_model    = "eval_accuracy",
    # — optimisation —
    learning_rate            = 2e-5,           # ↑ a bit, fewer params to tune
    weight_decay             = 0.01,
    gradient_accumulation_steps = 4,           # 4×4 GPU batch  = 16
    num_train_epochs         = 5,              # should converge; early-stop guard
    lr_scheduler_type        = "cosine",       # smoother than linear for small runs
    warmup_ratio             = 0.06,           # ≈ 300 warm-up steps
    # — hardware —
    per_device_train_batch_size = 4,
    per_device_eval_batch_size  = 8,           # bigger OK at eval
    fp16                      = True,
    # — misc —
    logging_strategy          = "steps",
    logging_steps             = 50,
    save_total_limit          = 3,
    seed                      = 42,
    report_to                 = "none",        # change if you use W&B / TensorBoard
    push_to_hub               = False,         # toggle later if you publish
)
# Data collator
data_collator = DefaultDataCollator(return_tensors="pt")

In [14]:
labels = rvl["label"]
print("Min label:", min(labels))
print("Max label:", max(labels))
print("Unique labels:", set(labels))

Min label: 0
Max label: 4
Unique labels: {0, 1, 2, 3, 4}


In [14]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DefaultDataCollator(),
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,0.541900,0.503387,0.841000,0.842479,0.841000,0.839606
500,0.375700,0.421162,0.871000,0.877401,0.871000,0.869710
750,0.269600,0.393870,0.892000,0.893155,0.892000,0.891691
1000,0.175200,0.367790,0.905000,0.905308,0.905000,0.904629
1250,0.174100,0.363010,0.904000,0.903748,0.904000,0.903382


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1731: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TrainOutput(global_step=1250, training_loss=0.4247925479888916, metrics={'train_runtime': 2860.6232, 'train_samples_per_second': 6.991, 'train_steps_per_second': 0.437, 'total_flos': 5308298342400000.0, 'train_loss': 0.4247925479888916, 'epoch': 5.0})

In [17]:
best_model = trainer.model
best_model.push_to_hub("adamadam111/layoutlmv3-docclass-finetuned-frz")



model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

[]

In [18]:
processor.push_to_hub("adamadam111/layoutlmv3-docclass-finetuned-frz")

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adamadam111/layoutlmv3-docclass-finetuned-frz/commit/3589b313a6f9b3d63851e1e2d8df2bc8f9b71a33', commit_message='Upload processor', commit_description='', oid='3589b313a6f9b3d63851e1e2d8df2bc8f9b71a33', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adamadam111/layoutlmv3-docclass-finetuned-frz', endpoint='https://huggingface.co', repo_type='model', repo_id='adamadam111/layoutlmv3-docclass-finetuned-frz'), pr_revision=None, pr_num=None)